Problem Statement

Tourists going to place like a Toronto cannot experience the real culture if they cannot eat the food they like in style of Tororto. This program will ask you about preferences of food and will recommend you the food you should eat along with location of restaurant.

Data Scraping and useage

Using fourSquare API, I will scrap the places along with their location, ask you some basic preferences and will recommend the restaurant the dish you must try by referencing it with other users with maximum intersection and giving their choice as your recomendation.

Step 1 : Know all the burroug and clean the datas

In [31]:
import requests
import pandas as pd
import folium
import numpy as np
import geopy.geocoders
from geopy.geocoders import Nominatim
import certifi
import time
import ssl
import csv

In [32]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
df['Borough'].astype(str)
df = df.drop(df[df['Borough'] == "Not assigned"].index)
arr = df[df['Neighbourhood'] == "Not assigned"].index
for i in arr:
    df.loc[i]['Neighbourhood'] = df.loc[i]['Borough']
df['Latitude'] = 0
df['Longitude'] = 0
df1 = pd.read_csv("Geospatial_Coordinates.csv")
for f in range(0, df1.shape[0]):
    pc = df1.iloc[f, 0]
    for i in range(0, df.shape[0]):
        temp = str(df.iloc[i, 0])
        if(temp == pc) : 
            df.iloc[i, 3] = df1.iloc[f, 1]
            df.iloc[i, 4] = df1.iloc[f, 2]
df.head()
#All values of Latitute and Longitude filled in df using csv provided

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [33]:
address = 'Toronto'
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
geopy.geocoders.options.default_ssl_context = ctx
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9.5)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
def get_likes(row):
    try:
        categories_list = row['likes']
    except:
        categories_list = row['venue.likes']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
import json
CLIENT_ID = 'AQJZAR2CDWN5F5SHA1KTVU1U3G1OLXYJVQ3G14H5MZG2Q5OA' # your Foursquare ID
CLIENT_SECRET = 'Y5RWWZ1UNAQZW01HYWP00KPIUKPDFDNVCI42IWMP1Q0SD4A4' # your Foursquare Secret
temp_latitude = df.iloc[0,3]
temp_longitude =df.iloc[0,4]
VERSION = '20180605' # Foursquare API version
radius = 500
limit = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    temp_latitude, 
    temp_longitude, 
    radius, 
    limit)
results = requests.get(url).json()
results

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=AQJZAR2CDWN5F5SHA1KTVU1U3G1OLXYJVQ3G14H5MZG2Q5OA&client_secret=Y5RWWZ1UNAQZW01HYWP00KPIUKPDFDNVCI42IWMP1Q0SD4A4&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1216f6160>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [45]:
import json
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    #Pass in list of names, latitude, longitude
    CLIENT_ID = 'AQJZAR2CDWN5F5SHA1KTVU1U3G1OLXYJVQ3G14H5MZG2Q5OA' # your Foursquare ID
    CLIENT_SECRET = 'Y5RWWZ1UNAQZW01HYWP00KPIUKPDFDNVCI42IWMP1Q0SD4A4' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
        limit = 100
        temp_latitude = lat
        temp_longitude = long
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            temp_latitude, 
            temp_longitude, 
            radius, 
            limit)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'])  for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
            'Neighbourhood Latitude', 
            'Neighbourhood Longitude', 
            'Venue',
            'Venue Latitude', 
            'Venue Longitude', 
            'Venue Category',
            'Likes', 
            'Ratings']
    return(nearby_venues)

In [46]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                                )
# with open ("torontovenues.csv", 'w') as f:
#     writer = csv.writer(f, delimiter = ",")
#     for data in range(1,toronto_venues.shape[0]):
#         writer.writerow(toronto_venues.iloc[data, :])
# toronto_venues.head()

Parkwoods


KeyboardInterrupt: 

In [77]:
df1 = pd.read_csv('torontovenues.csv')
df1

,Parkwoods,43.7532586,-79.52099940000001,KFC,43.75438666345904,-79.3330206627504,Fast Food Restaurant
0,Parkwoods,43.753259,-79.520999,Variety Store,43.751974,-79.333114,Food & Drink Shop
1,Victoria Village,43.725882,-79.520999,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
2,Victoria Village,43.725882,-79.520999,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,Victoria Village,43.725882,-79.520999,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.520999,The Frig,43.727051,-79.317418,French Restaurant
5,Victoria Village,43.725882,-79.520999,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Victoria Village,43.725882,-79.520999,Pizza Nova,43.725824,-79.312860,Pizza Place
7,Harbourfront,43.654260,-79.520999,Roselle Desserts,43.653447,-79.362017,Bakery
8,Harbourfront,43.654260,-79.520999,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,Harbourfront,43.654260,-79.520999,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center


In [73]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9.5)
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

NameError: name 'kclusters' is not defined